## Lens Generation for IDEAL surfaces

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import plotly
import sys
sys.path.append('/home/your_username/EncodingInformation/imager_experiments')
from lensless_helpers import *
from cleanplots import *


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

In [ ]:
#skimage resize
from skimage.transform import resize

## Render all the lenslets for each step

In [ ]:
# load the surfaces
all_lenslets = np.load('lenslet_surfaces.npy')
print(all_lenslets.shape, all_lenslets.dtype)

## Make lens renders

In [ ]:
for i in range(180):
    # Define the surface geometry
    x = np.linspace(-1, 1, 500)
    y = np.linspace(-1, 1, 500)
    x_grid, y_grid = np.meshgrid(y, x)
    r = np.sqrt(x_grid ** 2 + y_grid ** 2)
    z_diffuser = -15 * np.copy(all_lenslets[i]) 
    z_diffuser = resize(z_diffuser, (500, 500), anti_aliasing=True)
    z_diffuser_circle = np.copy(z_diffuser) # make a copy so that the edge code below can find the correct height
    z_diffuser_circle[r > 1] = np.nan
    opacity_mask = (r < 1).astype(np.float32)
    lighting = dict(ambient=0.1, fresnel=0, roughness=0.2, specular=.065) # specular will make it brighter/saturat
    lightposition=dict(x=5, y=5, z=-2)
    opacity=1
    # color_name= 'blue'
    color1 = 'rgb(80, 110, 255)'
    color2 = 'rgb(150, 180, 255)'
    # Create a 3D plot
    fig = go.Figure()
    fig.add_surface(
        x=x,
        y=y,
        z=z_diffuser_circle,
        colorscale=[[0, color1], [1, color2]], 
        showscale=False, 
        opacity=opacity,     
        lighting=lighting,
        lightposition=lightposition,
    )
    # Define the cylinder geometry
    thickness = 0.1
    theta = np.linspace(0, 2*np.pi, 500)
    z_cylinder = np.linspace(-.1, .1, 250)  # Adjust -1 and 1 to set the height of the cylinder
    theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
    x_cylinder = np.cos(theta_grid)
    y_cylinder = np.sin(theta_grid)
    # change the height to match the height of the phase mask
    for j in range(z_grid.shape[1]):
        # find the height of the phase mask at this xy location
        y_pos, x_pos = y_cylinder[0, j], x_cylinder[0, j]  # swap x_cylinder and y_cylinder to match x and y axes correctly
        # Convert from [-1, 1] to [0, size) and ensure index is within bounds
        x_index = np.clip(int((x_pos + 1) / 2 * (x.size - 1)), 0, x.size - 1)
        y_index = np.clip(int((y_pos + 1) / 2 * (y.size - 1)), 0, y.size - 1)
        # find the corresponding index in the phase mask
        phase_mask_height = z_diffuser[y_index, x_index]
        z_grid[:, j] = np.linspace(phase_mask_height, thickness, 250)  # Adjust the end height (.1) as needed
    # Assuming the cylinder's radius is 1
    color2 = 'rgb(120, 150, 255)'
    color1 = 'rgb(180, 210, 255)'
    fig.add_surface(
        x=x_cylinder, 
        y=y_cylinder, 
        z=z_grid, 
        colorscale=[[0, color1], [1, color2]],  # Change color as needed
        showscale=False, 
        opacity=opacity,
        # lighting=lighting,
        lightposition=lightposition
    )
    fig.update_layout(
        scene=dict(
            xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
            yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
            zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.1)
        ),
        paper_bgcolor=None,
        plot_bgcolor=None
    )
    
    # turn off axis labels
    fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
    # Update the camera to rotate the plot
    fig.update_layout(
        scene_camera=dict(
            eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
            up=dict(x=1, y=0, z=0)            # 'up' direction
        ),
    )
    fig.show()
    #fig.write_image('renders/{}_render.pdf'.format(i), scale=3)
    